In [6]:
%env BLINKA_U2IF=1
#%env BLINKA_MCP2221=1

# The Adafruit example says the above env setting will work but I've had to set it on the shell that launches Juptyer
#
# MANDATORY
#   Linux set BLINKA_MCP2221=1
#   Powershell $env:BLINKA_MCP2221=1
# or
#   Linux set BLINKA_U2IF=1
#   Powershell $env:BLINKA_U2IF=1
#
#
# Jupyter notebook demonstrating the APDS09960 proximity sensor
#
# Prerequisites:
# python3 -m pip install adafruit-circuitpython-apds9960
# python3 -m pip install ipyml
#
# add ../ to the path to pick up the lib directory with our modified library
import time

import board
import busio
import adafruit_apds9960.apds9960
#help(board)
# generic PICO UI2F SCL and SCA don't align with the Stemma connector on Keebs
#i2c = board.I2C(board.SCL, board.SDA)
# this is for the keebs board using the generic PCIO
i2c = busio.I2C(board.GP13, board.GP12)
sensor = adafruit_apds9960.apds9960.APDS9960(i2c)




env: BLINKA_U2IF=1


ValueError: I2C not found on specified pins.

In [ ]:
# warm up(?) don't remember why this is here
run_count = 500
print("starting run [", run_count, "]... ")
pre_run = time.perf_counter()
for run_num in range(run_count):
    tosser = gyro.gyro

post_run = time.perf_counter()
print(
    "take " + str(run_count) + " readings " + str(post_run - pre_run) + " secs"
)
# can be commented out
print(f"Snapshot of Angular Velocity (rad/s): {gyro.gyro_raw} - {gyro.gyro}")
print("ending run [", run_count, "]... ")


In [ ]:
# %matplotlib inline
# %matplotlib notebook
%matplotlib ipympl
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import datetime
import matplotlib.dates as mdates
from collections import deque

# How many sensor samples we want to store
HISTORY_SIZE = 25

# Pause re-sampling the sensor and drawing for INTERVAL seconds
INTERVAL = 0.01

# Deque for X-Axis (time)
x_vals = deque(maxlen=HISTORY_SIZE)

# Deque for Y-Axis (accelerometer readings)
gyro_x = deque(maxlen=HISTORY_SIZE)
gyro_y = deque(maxlen=HISTORY_SIZE)
gyro_z = deque(maxlen=HISTORY_SIZE)

# Create 3 side-by-side subplots
fig, (gyro1, gyro2, gyro3) = plt.subplots(1,3)

# Automatically adjust subplot parameters for nicer padding between plots
plt.tight_layout()

def animate(i):
    # Poll the Gyro
    gyro_data = gyro.gyro
    # Add the X/Y/Z values to the accel arrays
    gyro_x.append(gyro_data[0])
    gyro_y.append(gyro_data[1])
    gyro_z.append(gyro_data[2])

    # Grab the datetime, auto-range based on length of gyro_x array
    x_vals = [datetime.datetime.now() + datetime.timedelta(seconds=i) for i in range(len(gyro_x))]

    # Clear all axis
    gyro1.cla()
    gyro2.cla()
    gyro3.cla()

    # Set grid titles
    gyro1.set_title('X', fontsize=10)
    gyro2.set_title('Y', fontsize=10)
    gyro3.set_title('Z', fontsize=10)

    # Enable subplot grid lines
    gyro1.grid(True, linewidth=0.5, linestyle=':')
    gyro2.grid(True, linewidth=0.5, linestyle=':')
    gyro3.grid(True, linewidth=0.5, linestyle=':')

    # Rotate and align x-axis tick labels so they look better
    fig.autofmt_xdate()

    # Display the sub-plots
    gyro1.plot(x_vals, gyro_x, color='r')
    gyro2.plot(x_vals, gyro_y, color='g')
    gyro3.plot(x_vals, gyro_z, color='b')

    # Pause the plot for INTERVAL seconds
    plt.pause(INTERVAL)

# Update graph every 125ms
ani = FuncAnimation(fig, func=animate, frames=100, cache_frame_data=False)